In [1]:
from src.data_preparation.data_preparation import read_eeg_files
from scipy import signal
from src.algorithms.csp.CSP import CSP
import pywt
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
import numpy as np

Declaration of important values used during the pipeline 

In [2]:
TIME_LENGTH = 300
TIME_WINDOW = 300
EPOCH_SIZE = None
DATA_FOLDER = "data/bci-csp-based/bci-iii-dataset-iv-a"
CSP_COMPONENTS = 8
FS = 100
WAVELET = "coif1"
K_FOLD = 10

Create the subjects object and a dictionary to store the accuracies

In [8]:
subjects = range(1, 6)
subjects_set = set(subjects)
accuracies = {
    "GNB": np.zeros(K_FOLD),
    "SVM": np.zeros(K_FOLD),
    "LDA": np.zeros(K_FOLD)
}

Iterate on the subjects applying the algorithm, validating the results using the technique 10-fold cross-validation

In [9]:
# Load data
print("Loading data ...")
path_files = [(f"{DATA_FOLDER}/left-hand-subject-{subject}.csv",
               f"{DATA_FOLDER}/right-hand-subject-{subject}.csv")
              for subject in subjects]
data = read_eeg_files(path_files, TIME_LENGTH, TIME_WINDOW, EPOCH_SIZE)

# Pre-processing
print("Pre-processing ...")
print("Applying 5º order Butterworth bandpass filter (7-30 Hz)")
b, a = signal.butter(5, [7, 30], btype="bandpass", fs=FS)

data.left_data = signal.filtfilt(b, a, data.left_data, axis=1)
data.right_data = signal.filtfilt(b, a, data.right_data, axis=1)

print("Spatial-filtering ...")
data.X = np.concatenate((data.left_data, data.right_data))

csp = CSP(average_trial_covariance=False, n_components=CSP_COMPONENTS)
csp.fit(data.left_data, data.right_data)
data.Z = np.array([csp.project(x) for x in data.X])

# Feature extraction
print("Extracting features ...")
data.F = np.zeros((data.X.shape[0], 2, CSP_COMPONENTS))
for n_epoch in range(0, data.X.shape[0]):
    epoch = data.Z[n_epoch]

    # Calculate the wavelet features
    for n_feature in range(0, data.F.shape[2]):
        alpha_band, beta_band = pywt.dwt(epoch[:, n_feature], WAVELET)
        data.F[n_epoch, 0, n_feature] = np.sum(beta_band ** 2)

    # Calculate the frequency-domain features
    psd_window_size = 100
    psd_window_overlap = psd_window_size//2
    beta_freqs = range(13, 31)
    for n_feature in range(0, data.F.shape[2]):
        freq, psd = signal.welch(epoch[:, n_feature], fs=FS, window="hanning",
                                 nperseg=psd_window_size, noverlap=psd_window_overlap)
        data.F[n_epoch, 1, n_feature] = np.sum(psd[beta_freqs] ** 2)

len_features = data.F.shape[1] * data.F.shape[2]
data.F = np.reshape(data.F, newshape=(data.F.shape[0], len_features))

# Feature normalization
data.F = stats.zscore(data.F, axis=0)

# Classification
print("Classifying features ...")

cv = KFold(n_splits=K_FOLD, shuffle=True)
for (k, (train_index, test_index)) in enumerate(cv.split(data.F)):
    X_train, X_test = data.F[train_index], data.F[test_index]
    y_train, y_test = data.labels[train_index], data.labels[test_index]

    # GNB classifier
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    gnb_predictions = gnb.predict(X_test)
    gnb_accuracy = accuracy_score(y_test, gnb_predictions)
    print(f"GNB accuracy: {gnb_accuracy:.4f}")
    accuracies["GNB"][k] = gnb_accuracy

    # SVM classifier
    svm = SVC(C=.8, gamma="scale", kernel="rbf")
    svm.fit(X_train, y_train)
    svm_predictions = svm.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_predictions)
    print(f"SVM accuracy: {svm_accuracy:.4f}")
    accuracies["SVM"][k] = svm_accuracy

    # LDA classifier
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, y_train)
    lda_predictions = lda.predict(X_test)
    lda_accuracy = accuracy_score(y_test, lda_predictions)
    print(f"LDA accuracy: {lda_accuracy:.4f}")
    accuracies["LDA"][k] = lda_accuracy

print()

Loading data ...
Pre-processing ...
Applying 5º order Butterworth bandpass filter (7-30 Hz)
Spatial-filtering ...
Extracting features ...
Classifying features ...
GNB accuracy: 0.6643
SVM accuracy: 0.8500
LDA accuracy: 0.7786
GNB accuracy: 0.6929
SVM accuracy: 0.8286
LDA accuracy: 0.8000
GNB accuracy: 0.7071
SVM accuracy: 0.8000
LDA accuracy: 0.7714
GNB accuracy: 0.6929
SVM accuracy: 0.7571
LDA accuracy: 0.7571
GNB accuracy: 0.7143
SVM accuracy: 0.7929
LDA accuracy: 0.7643
GNB accuracy: 0.7214
SVM accuracy: 0.8000
LDA accuracy: 0.7857
GNB accuracy: 0.6786
SVM accuracy: 0.7857
LDA accuracy: 0.7357
GNB accuracy: 0.6929
SVM accuracy: 0.8143
LDA accuracy: 0.7500
GNB accuracy: 0.6929
SVM accuracy: 0.7500
LDA accuracy: 0.6857
GNB accuracy: 0.7500
SVM accuracy: 0.8214
LDA accuracy: 0.7857



Print the results obtained by the 10-fold cross-validation from each classification algorithm

In [12]:
for classifier in accuracies:
    print(classifier)
    for k, cv_accuracies in enumerate(accuracies[classifier]):
        acc_mean = np.mean(cv_accuracies)
        acc_std = np.std(cv_accuracies)
        print(f"\tK={k+1} average accuracy: {acc_mean:.4f} +/- {acc_std:.4f}")
    average_acc_mean = np.mean(accuracies[classifier])
    average_acc_std = np.std(accuracies[classifier])
    print(f"\tAverage accuracy: {average_acc_mean:.4f} +/- {average_acc_std:.4f}")


GNB
	K=1 average accuracy: 0.6643 +/- 0.0000
	K=2 average accuracy: 0.6929 +/- 0.0000
	K=3 average accuracy: 0.7071 +/- 0.0000
	K=4 average accuracy: 0.6929 +/- 0.0000
	K=5 average accuracy: 0.7143 +/- 0.0000
	K=6 average accuracy: 0.7214 +/- 0.0000
	K=7 average accuracy: 0.6786 +/- 0.0000
	K=8 average accuracy: 0.6929 +/- 0.0000
	K=9 average accuracy: 0.6929 +/- 0.0000
	K=10 average accuracy: 0.7500 +/- 0.0000
	Average accuracy: 0.7007 +/- 0.0227
SVM
	K=1 average accuracy: 0.8500 +/- 0.0000
	K=2 average accuracy: 0.8286 +/- 0.0000
	K=3 average accuracy: 0.8000 +/- 0.0000
	K=4 average accuracy: 0.7571 +/- 0.0000
	K=5 average accuracy: 0.7929 +/- 0.0000
	K=6 average accuracy: 0.8000 +/- 0.0000
	K=7 average accuracy: 0.7857 +/- 0.0000
	K=8 average accuracy: 0.8143 +/- 0.0000
	K=9 average accuracy: 0.7500 +/- 0.0000
	K=10 average accuracy: 0.8214 +/- 0.0000
	Average accuracy: 0.8000 +/- 0.0293
LDA
	K=1 average accuracy: 0.7786 +/- 0.0000
	K=2 average accuracy: 0.8000 +/- 0.0000
	K=3 avera